In [21]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import logging
import copy
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import kneighbors_graph
import torch.distributions.bernoulli as bernoulli

from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
import torch.distributions.bernoulli as bernoulli

import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils import data
from torch.autograd import Variable
import torch.nn.functional as F
from torch.nn import Parameter

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils

### Transfer Source Data & Model (Concare)

In [22]:
data_path = './data/Challenge/normalized_old/'
small_part = False
arg_timestep = 1.0
batch_size = 256
epochs = 100

In [23]:
# all_x = pickle.load(open(data_path + 'x.dat', 'rb'))
# all_y = pickle.load(open(data_path + 'y.dat', 'rb'))
# all_names = pickle.load(open(data_path + 'name.dat', 'rb'))
# # time = pickle.load(open(data_path + 'time', 'rb'))
# # weight = pickle.load(open(data_path + 'weight', 'rb'))
# static = pickle.load(open(data_path + 'demo.dat', 'rb'))
# mask_x = pickle.load(open(data_path + 'mask_x.dat', 'rb'))
# mask_demo = pickle.load(open(data_path + 'mask_demo.dat', 'rb'))
# all_x_len = [len(i) for i in all_x]

# print(all_x[0])
# print(mask_x[0])
# print(all_names[0])
# print(static[0])

In [24]:
train_x = pickle.load(open(data_path + 'train_x.dat', 'rb'))
train_y = pickle.load(open(data_path + 'train_y.dat', 'rb'))
train_names = pickle.load(open(data_path + 'train_names.dat', 'rb'))
train_static = pickle.load(open(data_path + 'train_static.dat', 'rb'))
train_x_len = pickle.load(open(data_path + 'train_x_len.dat', 'rb'))
train_mask_x = pickle.load(open(data_path + 'train_mask_x.dat', 'rb'))

dev_x = pickle.load(open(data_path + 'dev_x.dat', 'rb'))
dev_y = pickle.load(open(data_path + 'dev_y.dat', 'rb'))
dev_names = pickle.load(open(data_path + 'dev_names.dat', 'rb'))
dev_static = pickle.load(open(data_path + 'dev_static.dat', 'rb'))
dev_x_len = pickle.load(open(data_path + 'dev_x_len.dat', 'rb'))
dev_mask_x = pickle.load(open(data_path + 'dev_mask_x.dat', 'rb'))

test_x = pickle.load(open(data_path + 'test_x.dat', 'rb'))
test_y = pickle.load(open(data_path + 'test_y.dat', 'rb'))
test_names = pickle.load(open(data_path + 'test_names.dat', 'rb'))
test_static = pickle.load(open(data_path + 'test_static.dat', 'rb'))
test_x_len = pickle.load(open(data_path + 'test_x_len.dat', 'rb'))
test_mask_x = pickle.load(open(data_path + 'test_mask_x.dat', 'rb'))

print(len(train_x))
print(len(dev_x))
print(len(test_x))
print(sum(train_y)/len(train_y))
print(sum(dev_y)/len(dev_y))
print(sum(test_y)/len(test_y))

32268
4033
4035
0.07267261683401513
0.07265063228365981
0.07286245353159851


In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
# device = torch.device('cuda')
print("available device: {}".format(device))

available device: cuda:0


In [6]:
def get_loss(y_pred, y_true):
    loss = torch.nn.BCELoss()
    return loss(y_pred, y_true)

In [7]:
def get_re_loss(y_pred, y_true):
    loss = torch.nn.MSELoss()
    return loss(y_pred, y_true)

In [8]:
def pad_sents(sents, pad_token):

    sents_padded = []

    max_length = max([len(_) for _ in sents])
    for i in sents:
        padded = list(i) + [pad_token]*(max_length-len(i))
        sents_padded.append(np.array(padded))


    return np.array(sents_padded)

In [9]:
def batch_iter(x, y, mask, lens, batch_size, shuffle=False):
    """ Yield batches of source and target sentences reverse sorted by length (largest to smallest).
    @param data (list of (src_sent, tgt_sent)): list of tuples containing source and target sentence
    @param batch_size (int): batch size
    @param shuffle (boolean): whether to randomly shuffle the dataset
    """
    batch_num = math.ceil(len(x) / batch_size) # 向下取整
    index_array = list(range(len(x)))

    if shuffle:
        np.random.shuffle(index_array)

    for i in range(batch_num):
        indices = index_array[i * batch_size: (i + 1) * batch_size] #  fetch out all the induces
        
        examples = []
        for idx in indices:
            examples.append((x[idx], y[idx], mask[idx], lens[idx]))
       
        examples = sorted(examples, key=lambda e: len(e[0]), reverse=True)
    
        batch_x = [e[0] for e in examples]
        batch_y = [e[1] for e in examples]
        batch_mask_x = [e[2] for e in examples]
#         batch_name = [e[2] for e in examples]
        batch_lens = [e[3] for e in examples]

        yield batch_x, batch_y, batch_mask_x, batch_lens

In [10]:
def length_to_mask(length, max_len=None, dtype=None):
    """length: B.
    return B x max_len.
    If max_len is None, then max of length will be used.
    """
    assert len(length.shape) == 1, 'Length shape should be 1 dimensional.'
    max_len = max_len or length.max().item()
    mask = torch.arange(max_len, device=length.device,
                        dtype=length.dtype).expand(len(length), max_len) < length.unsqueeze(1)
    if dtype is not None:
        mask = torch.as_tensor(mask, dtype=dtype, device=length.device)
    return mask

In [11]:
class SingleAttention(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', demographic_dim=12, time_aware=False, use_demographic=False):
        super(SingleAttention, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim
        self.use_demographic = use_demographic
        self.demographic_dim = demographic_dim
        self.time_aware = time_aware

        # batch_time = torch.arange(0, batch_mask.size()[1], dtype=torch.float32).reshape(1, batch_mask.size()[1], 1)
        # batch_time = batch_time.repeat(batch_mask.size()[0], 1, 1)
        
        if attention_type == 'add':
            if self.time_aware == True:
                # self.Wx = nn.Parameter(torch.randn(attention_input_dim+1, attention_hidden_dim))
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
                self.Wtime_aware = nn.Parameter(torch.randn(1, attention_hidden_dim))
                nn.init.kaiming_uniform_(self.Wtime_aware, a=math.sqrt(5))
            else:
                self.Wx = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wt = nn.Parameter(torch.randn(attention_input_dim, attention_hidden_dim))
            self.Wd = nn.Parameter(torch.randn(demographic_dim, attention_hidden_dim))
            self.bh = nn.Parameter(torch.zeros(attention_hidden_dim,))
            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wd, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wx, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wt, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'mul':
            self.Wa = nn.Parameter(torch.randn(attention_input_dim, attention_input_dim))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        elif attention_type == 'concat':
            if self.time_aware == True:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim+1, attention_hidden_dim))
            else:
                self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))

            self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
            self.ba = nn.Parameter(torch.zeros(1,))
            
            nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
            nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        else:
            raise RuntimeError('Wrong attention type.')
        
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
    
    def forward(self, input, demo=None):
 
        batch_size, time_step, input_dim = input.size() # batch_size * time_step * hidden_dim(i)
        #assert(input_dim == self.input_dim)

        # time_decays = torch.zeros((time_step,time_step)).to(device)# t*t
        # for this_time in range(time_step):
        #     for pre_time in range(time_step):
        #         if pre_time > this_time:
        #             break
        #         time_decays[this_time][pre_time] = torch.tensor(this_time - pre_time, dtype=torch.float32).to(device)
        # b_time_decays = tile(time_decays, 0, batch_size).view(batch_size,time_step,time_step).unsqueeze(-1).to(device)# b t t 1

        time_decays = torch.tensor(range(time_step-1,-1,-1), dtype=torch.float32).unsqueeze(-1).unsqueeze(0).to(device)# 1*t*1
        b_time_decays = time_decays.repeat(batch_size,1,1)# b t 1
        
        if self.attention_type == 'add': #B*T*I  @ H*I
            q = torch.matmul(input[:,-1,:], self.Wt)# b h
            q = torch.reshape(q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            if self.time_aware == True:
                # k_input = torch.cat((input, time), dim=-1)
                k = torch.matmul(input, self.Wx)#b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
                time_hidden = torch.matmul(b_time_decays, self.Wtime_aware)#  b t h
            else:
                k = torch.matmul(input, self.Wx)# b t h
                # k = torch.reshape(k, (batch_size, 1, time_step, self.attention_hidden_dim)) #B*1*T*H
            if self.use_demographic == True:
                d = torch.matmul(demo, self.Wd) #B*H
                d = torch.reshape(d, (batch_size, 1, self.attention_hidden_dim)) # b 1 h
            h = q + k + self.bh # b t h
            if self.time_aware == True:
                h += time_hidden
            h = self.tanh(h) #B*T*H
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step))# b t
        elif self.attention_type == 'mul':
            e = torch.matmul(input[:,-1,:], self.Wa)#b i
            e = torch.matmul(e.unsqueeze(1), input.permute(0,2,1)).squeeze() + self.ba #b t
        elif self.attention_type == 'concat':
            q = input[:,-1,:].unsqueeze(1).repeat(1,time_step,1)# b t i
            k = input
            c = torch.cat((q, k), dim=-1) #B*T*2I
            if self.time_aware == True:
                c = torch.cat((c, b_time_decays), dim=-1) #B*T*2I+1
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        # e = torch.exp(e - torch.max(e, dim=-1, keepdim=True).values)
        
        # if self.attention_width is not None:
        #     if self.history_only:
        #         lower = torch.arange(0, time_step).to(device) - (self.attention_width - 1)
        #     else:
        #         lower = torch.arange(0, time_step).to(device) - self.attention_width // 2
        #     lower = lower.unsqueeze(-1)
        #     upper = lower + self.attention_width
        #     indices = torch.arange(0, time_step).unsqueeze(0).to(device)
        #     e = e * (lower <= indices).float() * (indices < upper).float()
        
        # s = torch.sum(e, dim=-1, keepdim=True)
        # mask = subsequent_mask(time_step).to(device) # 1 t t 下三角
        # scores = e.masked_fill(mask == 0, -1e9)# b t t 下三角
        a = self.softmax(e) #B*T
        v = torch.matmul(a.unsqueeze(1), input).squeeze() #B*I

        return v, a

class FinalAttentionQKV(nn.Module):
    def __init__(self, attention_input_dim, attention_hidden_dim, attention_type='add', dropout=None):
        super(FinalAttentionQKV, self).__init__()
        
        self.attention_type = attention_type
        self.attention_hidden_dim = attention_hidden_dim
        self.attention_input_dim = attention_input_dim


        self.W_q = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_k = nn.Linear(attention_input_dim, attention_hidden_dim)
        self.W_v = nn.Linear(attention_input_dim, attention_hidden_dim)

        self.W_out = nn.Linear(attention_hidden_dim, 1)

        self.b_in = nn.Parameter(torch.zeros(1,))
        self.b_out = nn.Parameter(torch.zeros(1,))

        nn.init.kaiming_uniform_(self.W_q.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_k.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_v.weight, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.W_out.weight, a=math.sqrt(5))

        self.Wh = nn.Parameter(torch.randn(2*attention_input_dim, attention_hidden_dim))
        self.Wa = nn.Parameter(torch.randn(attention_hidden_dim, 1))
        self.ba = nn.Parameter(torch.zeros(1,))
        
        nn.init.kaiming_uniform_(self.Wh, a=math.sqrt(5))
        nn.init.kaiming_uniform_(self.Wa, a=math.sqrt(5))
        
        self.dropout = nn.Dropout(p=dropout)
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, input):
 
        batch_size, time_step, input_dim = input.size() # batch_size * input_dim + 1 * hidden_dim(i)
        input_q = self.W_q(torch.mean(input,1)) # b h
        input_k = self.W_k(input)# b t h
        input_v = self.W_v(input)# b t h

        if self.attention_type == 'add': #B*T*I  @ H*I

            q = torch.reshape(input_q, (batch_size, 1, self.attention_hidden_dim)) #B*1*H
            h = q + input_k + self.b_in # b t h
            h = self.tanh(h) #B*T*H
            e = self.W_out(h) # b t 1
            e = torch.reshape(e, (batch_size, time_step))# b t

        elif self.attention_type == 'mul':
            q = torch.reshape(input_q, (batch_size, self.attention_hidden_dim, 1)) #B*h 1
            e = torch.matmul(input_k, q).squeeze()#b t
            
        elif self.attention_type == 'concat':
            q = input_q.unsqueeze(1).repeat(1,time_step,1)# b t h
            k = input_k
            c = torch.cat((q, k), dim=-1) #B*T*2I
            h = torch.matmul(c, self.Wh)
            h = self.tanh(h)
            e = torch.matmul(h, self.Wa) + self.ba #B*T*1
            e = torch.reshape(e, (batch_size, time_step)) # b t 
        
        a = self.softmax(e) #B*T
        if self.dropout is not None:
            a = self.dropout(a)
        v = torch.matmul(a.unsqueeze(1), input_v).squeeze() #B*I

        return v, a

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

def tile(a, dim, n_tile):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)])).to(device)
    return torch.index_select(a, dim, order_index).to(device)

class PositionwiseFeedForward(nn.Module): # new added
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x)))), None

# class PositionwiseFeedForwardConv(nn.Module):

#     def __init__(self, model_dim=512, ffn_dim=2048, dropout=0.0):
#         super(PositionalWiseFeedForward, self).__init__()
#         self.w1 = nn.Conv1d(model_dim, ffn_dim, 1)
#         self.w2 = nn.Conv1d(model_dim, ffn_dim, 1)
#         self.dropout = nn.Dropout(dropout)
#         self.layer_norm = nn.LayerNorm(model_dim)

#     def forward(self, x):
#         output = x.transpose(1, 2)
#         output = self.w2(F.relu(self.w1(output)))
#         output = self.dropout(output.transpose(1, 2))

#         # add residual and norm layer
#         output = self.layer_norm(x + output)
#         return output

class PositionalEncoding(nn.Module): # new added / not use anymore
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=400):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0., max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0., d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], 
                         requires_grad=False)
        return self.dropout(x)

def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0 # 下三角矩阵

def attention(query, key, value, mask=None, dropout=None):
    "Compute 'Scaled Dot Product Attention'"
    d_k = query.size(-1)# b h t d_k
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k) # b h t t
    if mask is not None:# 1 1 t t
        scores = scores.masked_fill(mask == 0, -1e9)# b h t t 下三角
    p_attn = F.softmax(scores, dim = -1)# b h t t
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn # b h t v (d_k) 
    
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, self.d_k * self.h), 3)
        self.final_linear = nn.Linear(d_model, d_model)
        self.attn = None
        self.mask = None
        self.dropout = nn.Dropout(p=dropout)
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1) # 1 1 t t

        nbatches = query.size(0)# b
        input_dim = query.size(1)# i+1
        feature_dim = query.size(-1)# i+1

        #input size -> # batch_size * d_input * hidden_dim
        
        # d_model => h * d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))] # b num_head d_input d_k
        
        
        querys = []
        keys = []
        values = []
        for i in range(self.h):
            querys.append(query[:,i].unsqueeze(1))
            keys.append(key[:,i].unsqueeze(1))
            values.append(value[:,i].unsqueeze(1))
            
        
       
        if self.training == True:

            x, attn = attention(querys[0], keys[0], values[0], mask=mask, 
                                     dropout=self.dropout)# b num_head d_input d_v (d_k) 


            self.attn = attn
    #         self.mask = mask
            attn1 = torch.mean(attn[:,0],1)
    #         print(attn1.shape) # 256, 153
    #         attn1_p = attn1/torch.sum(attn1,1).unsqueeze(1)

            attn1_p = 0.1*self.sigmoid(lNorm(attn1)*10)
    #         print(attn1_p.shape)


            dis_b1 = bernoulli.Bernoulli(attn1_p)
            to_mask1 = 1 - dis_b1.sample().to(device)
            mask1 = to_mask1.unsqueeze(1).repeat(1,input_dim,1).unsqueeze(1)
    #             print(attn1_p)
    #             print(mask1)

    #         mask1 = torch.ones((nbatches,input_dim,2*input_dim-1)).to(device)
    #         mask1[0,0,attn1:attn1+1,:] = 0
    #         mask1[0,0,:,attn1:attn1+1] = 0

            tempx, attn = attention(querys[1], keys[1], values[1], mask=mask1, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            self.mask = mask1
            x = torch.cat((x, tempx),1)

            attn2 = attn1 + torch.mean(attn[:,0],1)
    #         attn2_p = attn2/torch.sum(attn2,1).unsqueeze(1)
            attn2_p = 0.1*self.sigmoid(lNorm(attn2)*10)

            dis_b2 = bernoulli.Bernoulli(attn2_p)
            to_mask2 = 1 - dis_b2.sample().to(device)
            mask2 = to_mask2.unsqueeze(1).repeat(1,input_dim,1).unsqueeze(1)

            tempx, attn = attention(querys[2], keys[2], values[2], mask=mask2, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            self.mask = torch.cat((self.mask, mask2), 1)
            x = torch.cat((x, tempx),1)

    #         attn3 = torch.mean(attn[:,0],0)
            attn3 = attn2 + torch.mean(attn[:,0],1)
    #         attn3_p = attn3/torch.sum(attn3,1).unsqueeze(1)
            attn3_p = 0.1*self.sigmoid(lNorm(attn3)*10)

            dis_b3 = bernoulli.Bernoulli(attn3_p)
            to_mask3 = 1 - dis_b3.sample().to(device)
            mask3 = to_mask3.unsqueeze(1).repeat(1,input_dim,1).unsqueeze(1)


            tempx, attn = attention(querys[3], keys[3], values[3], mask=mask3, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            self.mask = torch.cat((self.mask, mask3), 1)

    #         attn4 = torch.mean(attn[0,0],0)
            x = torch.cat((x, tempx),1)
    
        

        if self.training == False:
# #             print(1)
            
            x, attn = attention(querys[0], keys[0], values[0], mask=None, 
                                     dropout=self.dropout)# b num_head d_input d_v (d_k) 


            self.attn = attn

            tempx, attn = attention(querys[1], keys[1], values[1], mask=None, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            x = torch.cat((x, tempx),1)



            tempx, attn = attention(querys[2], keys[2], values[2], mask=None, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)
            x = torch.cat((x, tempx),1)




            tempx, attn = attention(querys[3], keys[3], values[3], mask=None, 
                                     dropout=self.dropout)
            self.attn = torch.cat((self.attn, attn), 1)

    #         attn4 = torch.mean(attn[0,0],0)
            x = torch.cat((x, tempx),1)
        
#         kl = nn.functional.kl_div(self.softmax(attn3).log(), self.softmax(attn4), size_average=None, reduce=None, reduction='mean') + \
#             nn.functional.kl_div(self.softmax(attn4).log(), self.softmax(attn3), size_average=None, reduce=None, reduction='mean')
#         torch.nn.functional.kl_div(attn3, attn4, reduction='batchmean') + torch.nn.functional.kl_div(attn4, attn3, reduction='batchmean')

        #Str DeCov
        # DeCov_contexts = x.transpose(1, 2).transpose(0, 1).transpose(1, 2).transpose(2, 3)#d_input num_head d_v b (d_k) 
        # DeCov_between_loss = torch.tensor(0.0, dtype = torch.float32, device = device)
        # DeCov_inside_loss = torch.tensor(0.0, dtype = torch.float32, device = device)
        # for input_idx in range(input_dim):
        #     current_input = DeCov_contexts[input_idx,:,:,:]
        #     for i in range(self.h):
        #         for j in range(self.h):
        #             if i == j:
        #                 covs = cov(current_input[i,:,:])
        #                 DeCov_inside_loss += 0.5 * (torch.norm(covs, p = 'fro')**2)  
        #                 #print(DeCov_inside_loss)
        #             else:
        #                 Vstack_context = torch.cat((current_input[i,:,:],current_input[j,:,:]) ,0)# 2*d_v b
        #                 assert(Vstack_context.size(0) == 2*self.d_k)
        #                 assert(Vstack_context.size(1) == nbatches)
        #                 overflow_covs = cov(Vstack_context)[self.d_k:,:]
        #                 covs = overflow_covs[:,:self.d_k]
        #                 DeCov_between_loss += 0.5 * (torch.norm(covs, p = 'fro')**2) 
        #                 #print(DeCov_between_loss)
        # DeCov_loss = DeCov_between_loss + 0.1 * DeCov_inside_loss
        #print(DeCov_loss)


      
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)# batch_size * d_input * hidden_dim

        #DeCov 
#         DeCov_contexts = x.transpose(0, 1).transpose(1, 2) # I+1 H B
# #         print(DeCov_contexts.shape)
#         Covs = cov(DeCov_contexts[0,:,:])
#         DeCov_loss = 0.5 * (torch.norm(Covs, p = 'fro')**2 - torch.norm(torch.diag(Covs))**2 ) 
#         for i in range(feature_dim -1+1):
#             Covs = cov(DeCov_contexts[i+1,:,:])
#             DeCov_loss += 0.5 * (torch.norm(Covs, p = 'fro')**2 - torch.norm(torch.diag(Covs))**2 ) 

#         print(DeCov_loss)
#         print(kl)
        
        return self.final_linear(x), torch.zeros(1).to(device)

        


class LayerNorm(nn.Module):
    def __init__(self, features, eps=1e-7):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2
    
def lNorm(x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True)
    eps = 1e-7
    return (x - mean) / (std + eps)

def cov(m, y=None):
    if y is not None:
        m = torch.cat((m, y), dim=0)
    m_exp = torch.mean(m, dim=1)
    x = m - m_exp[:, None]
    cov = 1 / (x.size(1) - 1) * x.mm(x.t())
    return cov

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        returned_value = sublayer(self.norm(x))
        return x + self.dropout(returned_value[0]) , returned_value[1]

class vanilla_transformer_encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, d_model,  MHD_num_head, d_ff, output_dim, keep_prob=0.5):
        super(vanilla_transformer_encoder, self).__init__()

        # hyperparameters
        self.input_dim = input_dim  
        self.hidden_dim = hidden_dim  # d_model
        self.d_model = d_model
        self.MHD_num_head = MHD_num_head
        self.d_ff = d_ff
        self.output_dim = output_dim
        self.keep_prob = keep_prob

        # layers
        self.PositionalEncoding = PositionalEncoding(self.d_model, dropout = 0, max_len = 400)

        self.GRUs = clones(nn.GRU(1, self.hidden_dim, batch_first = True), self.input_dim)
#         self.Convs = clones(nn.Sequential(         
#             nn.Conv1d(
#                 in_channels=1,              # input height
#                 out_channels=self.hidden_dim,            # n_filters
#                 kernel_size=4,              # filter size
#                 stride=1,                   # filter movement/step
# #                 padding=2, 
# #                 dilation = 2,# if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
#             ),                              # output shape (16, 28, 28)
#             nn.ReLU(),                      # activation
#           nn.MaxPool1d(1000, stride=None)
            
#         ), self.input_dim)
        
#         self.DConvs = clones(nn.Sequential(         
#             nn.Conv1d(
#                 in_channels=1,              # input height
#                 out_channels=self.hidden_dim,            # n_filters
#                 kernel_size=4,
#                 dilation = 2,# filter size
#                 stride=1,                   # filter movement/step
# #                 padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
#             ),                              # output shape (16, 28, 28)
#             nn.ReLU(),                      # activation
#           nn.MaxPool1d(1000, stride=None)
            
#         ), self.input_dim)
        
        
#         self.LastStepAttentions = clones(SingleAttention(self.hidden_dim, 8, attention_type='concat', demographic_dim=12, time_aware=True, use_demographic=False),self.input_dim)
        
        self.FinalAttentionQKV = FinalAttentionQKV(self.hidden_dim, self.hidden_dim, attention_type='mul',dropout = 1 - self.keep_prob)

        self.MultiHeadedAttention = MultiHeadedAttention(self.MHD_num_head, self.d_model,dropout = 1 - self.keep_prob)
        self.SublayerConnection = SublayerConnection(self.d_model, dropout = 1 - self.keep_prob)

        self.PositionwiseFeedForward = PositionwiseFeedForward(self.d_model, self.d_ff, dropout=0.1)

        self.demo_proj_main = nn.Linear(12, self.hidden_dim)
        self.demo_proj = nn.Linear(12, self.hidden_dim)
        self.output = nn.Linear(self.hidden_dim, self.output_dim)

        self.dropout = nn.Dropout(p = 1 - self.keep_prob)
        self.tanh=nn.Tanh()
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()
        self.relu=nn.ReLU()

    def forward(self, input, lens):
        # input shape [batch_size, timestep, feature_dim]
#         demo_main = self.tanh(self.demo_proj_main(demo_input)).unsqueeze(1)# b hidden_dim
        
        batch_size = input.size(0)
        time_step = input.size(1)
        feature_dim = input.size(2)
        assert(feature_dim == self.input_dim)# input Tensor : 256 * 48 * 76
        assert(self.d_model % self.MHD_num_head == 0)

     
        GRU_embeded_input = self.GRUs[0](pack_padded_sequence(input[:,:,0].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h

        for i in range(feature_dim-1):
            embeded_input = self.GRUs[i+1](pack_padded_sequence(input[:,:,i+1].unsqueeze(-1), lens, batch_first=True))[1].squeeze().unsqueeze(1) # b 1 h
            GRU_embeded_input = torch.cat((GRU_embeded_input, embeded_input), 1)
            
#         Conv_embeded_input = self.Convs[0](input[:,:,0].unsqueeze(1)).squeeze(-1).unsqueeze(1) # b 1 h
# #         print(Conv_embeded_input.shape)
#         for i in range(feature_dim-1):
#             embeded_input = self.Convs[i+1](input[:,:,i+1].unsqueeze(1)).squeeze(-1).unsqueeze(1) # b 1 h
#             Conv_embeded_input = torch.cat((Conv_embeded_input, embeded_input), 1)
        
# # #         print(GRU_embeded_input.shape)
# # #         print(Conv_embeded_input.shape)
#         GRU_embeded_input = torch.cat((GRU_embeded_input, Conv_embeded_input), 1)# b i+1 h
    
#         Conv_embeded_input = self.DConvs[0](input[:,:,0].unsqueeze(1)).squeeze(-1).unsqueeze(1) # b 1 h
# #         print(Conv_embeded_input.shape)
#         for i in range(feature_dim-1):
#             embeded_input = self.DConvs[i+1](input[:,:,i+1].unsqueeze(1)).squeeze(-1).unsqueeze(1) # b 1 h
#             Conv_embeded_input = torch.cat((Conv_embeded_input, embeded_input), 1)
            
#         GRU_embeded_input = torch.cat((GRU_embeded_input, Conv_embeded_input), 1)# b i+1 h
            
#         print( GRU_embeded_input.shape)
#         GRU_embeded_input =  Conv_embeded_input

#         GRU_embeded_input = torch.cat((GRU_embeded_input, demo_main), 1)# b i+1 h
        posi_input = self.dropout( GRU_embeded_input) # batch_size * d_input * hidden_dim


        #mask = subsequent_mask(time_step).to(device) # 1 t t 下三角 N to 1任务不用mask
        contexts = self.SublayerConnection(posi_input, lambda x: self.MultiHeadedAttention(posi_input, posi_input, posi_input, None))# # batch_size * d_input * hidden_dim
    
        DeCov_loss = contexts[1]
        contexts = contexts[0]

        contexts = self.SublayerConnection(contexts, lambda x: self.PositionwiseFeedForward(contexts))[0]# # batch_size * d_input * hidden_dim
        #contexts = contexts.view(batch_size, feature_dim * self.hidden_dim)#
        # contexts = torch.matmul(self.Wproj, contexts) + self.bproj
        # contexts = contexts.squeeze()
        # demo_key = self.demo_proj(demo_input)# b hidden_dim
        # demo_key = self.relu(demo_key)
        # input_dim_scores = torch.matmul(contexts, demo_key.unsqueeze(-1)).squeeze() # b i
        # input_dim_scores = self.dropout(self.sigmoid(input_dim_scores)).unsqueeze(1)# b i
        
        # weighted_contexts = torch.matmul(input_dim_scores, contexts).squeeze()
#         print(contexts.shape)

        weighted_contexts = self.FinalAttentionQKV(contexts)[0]
        output = self.output(self.dropout(weighted_contexts))# b 1
        output = self.sigmoid(output)
#         print(weighted_contexts.shape)
          
        return output, DeCov_loss  ,weighted_contexts
    #, self.MultiHeadedAttention.attn



In [12]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED) #numpy
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED) # cpu
torch.cuda.manual_seed(RANDOM_SEED) #gpu
torch.backends.cudnn.deterministic=True # cudnn
    
epochs = 150
batch_size = 256
input_dim = 33
hidden_dim = 32
d_model = 32
MHD_num_head = 4
d_ff = 64
output_dim = 1

model = vanilla_transformer_encoder(input_dim = input_dim, hidden_dim = hidden_dim, d_model = d_model,  MHD_num_head = MHD_num_head , d_ff = d_ff, output_dim = output_dim).to(device)
# input_dim, d_model, d_k, d_v, MHD_num_head, d_ff, output_dim
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [13]:
total_train_loss = []
total_valid_loss = []
global_best = 0
auroc = []
auprc = []
minpse = []
history = []

pad_token = np.zeros(33)
# begin_time = time.time()
best_auroc = 0
best_auprc = 0
best_sum = 0

    
file_name = './model/concare-mean-challenge-gru-newnormboost0.1'

for each_epoch in range(epochs):

    epoch_loss = []
    counter_batch = 0
    model.train()  

    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(train_x, train_y, train_mask_x, train_x_len, batch_size, shuffle=True)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

#        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt, decov_loss, _ = model(batch_x, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss + 1000 * decov_loss

        loss = model_loss

        epoch_loss.append(BCE_Loss.cpu().detach().numpy())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 20)
        optimizer.step()

        if step % 20 == 0:
            print('Epoch %d Batch %d: Train Loss = %.4f'%(each_epoch, step, loss.cpu().detach().numpy()))

    epoch_loss = np.mean(epoch_loss)
    total_train_loss.append(epoch_loss)

    #Validation
    y_true = []
    y_pred = []
    with torch.no_grad():
        model.eval()
        valid_loss = []
        valid_true = []
        valid_pred = []
        for batch_x, batch_y, batch_mask_x, batch_lens in batch_iter(dev_x, dev_y, dev_mask_x, dev_x_len, batch_size):
            batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
            batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
            batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
            batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)
#            masks = length_to_mask(batch_lens).unsqueeze(-1).float()


            opt, decov_loss, _ = model(batch_x, batch_lens)

            BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#                 REC_Loss = F.mse_loss(recon, batch_x, reduction='mean').to(device)

            valid_loss.append(BCE_Loss.cpu().detach().numpy())

            y_pred += list(opt.cpu().detach().numpy().flatten())
            y_true += list(batch_y.cpu().numpy().flatten())

        valid_loss = np.mean(valid_loss)
        total_valid_loss.append(valid_loss)
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())
        ret = metrics.print_metrics_binary(y_true, y_pred,verbose = 0)
        history.append(ret)
        #print()

        print('Epoch %d: Loss = %.4f Valid loss = %.4f roc = %.4f'%(each_epoch, total_train_loss[-1], total_valid_loss[-1], ret['auroc']))
        metrics.print_metrics_binary(y_true, y_pred)

        cur_auroc = ret['auroc']
        if cur_auroc > best_auroc:
            best_auroc = cur_auroc
#             best_auprc = ret['auprc']
#             best_minpse = ret['minpse']
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name)
            print('------------ Save best model - AUROC: %.4f ------------'%cur_auroc) 
        
        cur_prc = ret['auprc']
        cur_minpse = ret['minpse']
        
        if cur_prc > best_auprc:
            best_auprc = cur_prc
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name+"prc")
            print('\n------------ Save best-prc model ------------\n')

        cur_sum = cur_auroc + cur_prc + cur_minpse
    #     print(cur_sum)
        if cur_sum > best_sum:
            best_sum = cur_sum
            state = {
                'net': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'epoch': each_epoch
            }
            torch.save(state, file_name+"sum")
            print('\n------------ Save best-sum model ------------\n')

print('auroc %.4f'%(best_auroc))
print('auprc %.4f'%(best_auprc))
print('minpse %.4f'%(best_minpse))  

/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 0 Batch 0: Train Loss = 0.6265
Epoch 0 Batch 20: Train Loss = 0.3715
Epoch 0 Batch 40: Train Loss = 0.3228
Epoch 0 Batch 60: Train Loss = 0.3644
Epoch 0 Batch 80: Train Loss = 0.2476
Epoch 0 Batch 100: Train Loss = 0.2662
Epoch 0 Batch 120: Train Loss = 0.2635


/home/ubicomp/choczhang/EMR/challenge/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1]) # TP/TP+FP = PPV = precision
/home/ubicomp/choczhang/EMR/challenge/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1]) # TP/TP+FP = PPV = precision


Epoch 0: Loss = 0.3022 Valid loss = 0.2574 roc = 0.7286
confusion matrix:
[[3918    0]
 [ 308    0]]
accuracy = 0.9271178245544434
precision class 0 = 0.9271178245544434
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.7285749918789735
AUC of PRC = 0.2356456412611008
min(+P, Se) = 0.3215434083601286
f1_score = nan
------------ Save best model - AUROC: 0.7286 ------------

------------ Save best-prc model ------------


------------ Save best-sum model ------------



/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


...

/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 148 Batch 0: Train Loss = 0.1026
Epoch 148 Batch 20: Train Loss = 0.0594
Epoch 148 Batch 40: Train Loss = 0.1065
Epoch 148 Batch 60: Train Loss = 0.0700
Epoch 148 Batch 80: Train Loss = 0.0717
Epoch 148 Batch 100: Train Loss = 0.0779
Epoch 148 Batch 120: Train Loss = 0.1139
Epoch 148: Loss = 0.0954 Valid loss = 0.1270 roc = 0.9497
confusion matrix:
[[3858   60]
 [ 111  197]]
accuracy = 0.9595361948013306
precision class 0 = 0.9720332622528076
precision class 1 = 0.7665369510650635
recall class 0 = 0.9846860766410828
recall class 1 = 0.6396104097366333
AUC of ROC = 0.9496993562843488
AUC of PRC = 0.7679970264954991
min(+P, Se) = 0.7087378640776699
f1_score = 0.6973451389468387


/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch 149 Batch 0: Train Loss = 0.0979
Epoch 149 Batch 20: Train Loss = 0.0820
Epoch 149 Batch 40: Train Loss = 0.0685
Epoch 149 Batch 60: Train Loss = 0.0824
Epoch 149 Batch 80: Train Loss = 0.0712
Epoch 149 Batch 100: Train Loss = 0.0434
Epoch 149 Batch 120: Train Loss = 0.1286
Epoch 149: Loss = 0.0956 Valid loss = 0.1247 roc = 0.9502
confusion matrix:
[[3887   31]
 [ 130  178]]
accuracy = 0.9619024991989136
precision class 0 = 0.9676375389099121
precision class 1 = 0.8516746163368225
recall class 0 = 0.9920877814292908
recall class 1 = 0.5779221057891846
AUC of ROC = 0.9501849605218671
AUC of PRC = 0.7720148495879767
min(+P, Se) = 0.7077922077922078
f1_score = 0.688588048117687
auroc 0.9515
auprc 0.7828


NameError: name 'best_minpse' is not defined

In [14]:
# ####Plot Loss
# import matplotlib.pyplot as plt
# fig = plt.figure()
# ax1 = fig.add_subplot(1, 1, 1)
# x_axis = np.arange(1, 151)
# ax1.plot(x_axis, total_train_loss, c='red', label='train')
# ax1.plot(x_axis, total_valid_loss, c='blue', label='valid')
# plt.legend(loc='best')
# plt.show()

In [15]:
checkpoint = torch.load(file_name+'sum')
save_epoch = checkpoint['epoch']
print("last saved model is in epoch {}".format(save_epoch))
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()


last saved model is in epoch 132


vanilla_transformer_encoder(
  (PositionalEncoding): PositionalEncoding(
    (dropout): Dropout(p=0)
  )
  (GRUs): ModuleList(
    (0): GRU(1, 32, batch_first=True)
    (1): GRU(1, 32, batch_first=True)
    (2): GRU(1, 32, batch_first=True)
    (3): GRU(1, 32, batch_first=True)
    (4): GRU(1, 32, batch_first=True)
    (5): GRU(1, 32, batch_first=True)
    (6): GRU(1, 32, batch_first=True)
    (7): GRU(1, 32, batch_first=True)
    (8): GRU(1, 32, batch_first=True)
    (9): GRU(1, 32, batch_first=True)
    (10): GRU(1, 32, batch_first=True)
    (11): GRU(1, 32, batch_first=True)
    (12): GRU(1, 32, batch_first=True)
    (13): GRU(1, 32, batch_first=True)
    (14): GRU(1, 32, batch_first=True)
    (15): GRU(1, 32, batch_first=True)
    (16): GRU(1, 32, batch_first=True)
    (17): GRU(1, 32, batch_first=True)
    (18): GRU(1, 32, batch_first=True)
    (19): GRU(1, 32, batch_first=True)
    (20): GRU(1, 32, batch_first=True)
    (21): GRU(1, 32, batch_first=True)
    (22): GRU(1, 32, batc

In [16]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_mask_x, batch_lens) in enumerate(batch_iter(test_x, test_y, test_mask_x, test_x_len, batch_size, shuffle=True)):  
        optimizer.zero_grad()
        batch_x = torch.tensor(pad_sents(batch_x, pad_token), dtype=torch.float32).to(device)
        batch_y = torch.tensor(batch_y, dtype=torch.float32).to(device)
        batch_lens = torch.tensor(batch_lens, dtype=torch.float32).to(device).int()
        batch_mask_x = torch.tensor(pad_sents(batch_mask_x, pad_token), dtype=torch.float32).to(device)

        masks = length_to_mask(batch_lens).unsqueeze(-1).float()

        opt, decov_loss, _ = model(batch_x, batch_lens)

        BCE_Loss = get_loss(opt, batch_y.unsqueeze(-1))
#             REC_Loss = F.mse_loss(masks * recon, masks * batch_x, reduction='mean').to(device)

        model_loss =  BCE_Loss 

        loss = model_loss
        batch_loss.append(loss.cpu().detach().numpy())
        if step % 20 == 0:
            print('Batch %d: Test Loss = %.4f'%(step, loss.cpu().detach().numpy()))
        y_pred += list(opt.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\n==>Predicting on test")
print('Test Loss = %.4f'%(np.mean(np.array(batch_loss))))
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

/home/ubicomp/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:188: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


Batch 0: Test Loss = 0.0990

==>Predicting on test
Test Loss = 0.1233
confusion matrix:
[[3669   72]
 [  94  200]]
accuracy = 0.9588599801063538
precision class 0 = 0.9750199317932129
precision class 1 = 0.7352941036224365
recall class 0 = 0.9807537794113159
recall class 1 = 0.680272102355957
AUC of ROC = 0.9557750392324799
AUC of PRC = 0.7902974773219105
min(+P, Se) = 0.6938775510204082
f1_score = 0.7067137709400907


In [20]:
# Bootstrap
N = len(y_true)
N_idx = np.arange(N)
K = 1000

auroc = []
auprc = []
minpse = []


acc = []# TN+TP/ALL
pre0 = []
pre1 = [] # TP/TP+FP = PPV = precision
rec0 = [] # TN/TN+FP = TNR = specificity = selectivity
rec1 = []# TP/TP+FN = recall = TPR = sensitivity
f1 = []


for i in range(K):
    boot_idx = np.random.choice(N_idx, N, replace=True)
    boot_true = np.array(y_true)[boot_idx]
    boot_pred = y_pred[boot_idx, :]
    test_ret = metrics.print_metrics_binary(boot_true, boot_pred, verbose=0)
    auroc.append(test_ret['auroc'])
    auprc.append(test_ret['auprc'])
    minpse.append(test_ret['minpse'])
    acc.append(test_ret['acc'])
    pre0.append(test_ret['prec0'])
    pre1.append(test_ret['prec1'])
    rec0.append(test_ret['rec0'])
    rec1.append(test_ret['rec1'])
    f1.append(test_ret['f1_score'])
#     print('%d/%d'%(i+1,K))
    
auroc = np.sort(auroc)
auprc = np.sort(auprc)
minpse = np.sort(minpse)

acc = np.sort(acc)
pre0 = np.sort(pre0)
pre1 = np.sort(pre1)
rec0 = np.sort(rec0)
rec1 = np.sort(rec1)
f1 = np.sort(f1)

print('auroc %.4f(%.4f, %.4f-%.4f)'%(np.mean(auroc), np.std(auroc), auroc[int(K*0.025)], auroc[int(K*0.975)]))
print('auprc %.4f(%.4f, %.4f-%.4f)'%(np.mean(auprc), np.std(auprc), auprc[int(K*0.025)], auprc[int(K*0.975)]))
print('minpse %.4f(%.4f, %.4f-%.4f)'%(np.mean(minpse), np.std(minpse), minpse[int(K*0.025)], minpse[int(K*0.975)]))
print('f1 %.4f(%.4f, %.4f-%.4f)'%(np.mean(f1), np.std(f1), f1[int(K*0.025)], f1[int(K*0.975)]))
print('acc %.4f(%.4f, %.4f-%.4f)'%(np.mean(acc), np.std(acc), acc[int(K*0.025)], acc[int(K*0.975)]))
print('pre0 %.4f(%.4f, %.4f-%.4f)'%(np.mean(pre0), np.std(pre0), pre0[int(K*0.025)], pre0[int(K*0.975)]))
print('pre1 %.4f(%.4f, %.4f-%.4f)'%(np.mean(pre1), np.std(pre1), pre1[int(K*0.025)], pre1[int(K*0.975)]))

print('rec0 %.4f(%.4f, %.4f-%.4f)'%(np.mean(rec0), np.std(rec0), rec0[int(K*0.025)], rec0[int(K*0.975)]))
print('rec1 %.4f(%.4f, %.4f-%.4f)'%(np.mean(rec1), np.std(rec1), rec1[int(K*0.025)], rec1[int(K*0.975)]))


auroc 0.9559(0.0061, 0.9435-0.9669)
auprc 0.7905(0.0216, 0.7463-0.8314)
minpse 0.6976(0.0240, 0.6496-0.7416)
f1 0.7063(0.0227, 0.6626-0.7481)
acc 0.9589(0.0031, 0.9524-0.9651)
pre0 0.9750(0.0026, 0.9700-0.9800)
pre1 0.7366(0.0278, 0.6831-0.7909)
rec0 0.9809(0.0023, 0.9764-0.9853)
rec1 0.6791(0.0283, 0.6219-0.7338)
